In [2]:
#TUNING THE PARAMETERS
# we may want to improve our algos, we had to split data into like biased and unbiased
# use the biased set for paramters will be used in grid search 
# other set will be used for unbiased estimation

In [7]:
import random
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import KFold, GridSearchCV

In [10]:
#load the data
data = Dataset.load_builtin()#default ml-100k
raw_ratings = data.raw_ratings

#shuffle the ratings
random.shuffle(raw_ratings)

# split the data like train_test_split
# A = 0.75, B=0.25 of the data

threshold = int(0.75 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]
data.raw_ratings = A_raw_ratings # data becomes set A

# Grid search implementation, it allows to selecet wich parameters are best to train the algorithm
# set the parameters range as param_grid
print("----GRID SEARCH----")
param_grid = {'n_epochs': [5,10], 'lr_all':[0.002, 0.005]}
grid_search = GridSearchCV(SVD, param_grid=param_grid, measures=['rmse'], cv=3)
grid_search.fit(data)

algo = grid_search.best_estimator['rmse']
print(algo)

----GRID SEARCH----


In [11]:
#retrain the on whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

#compute the biased whole set A
predictions = algo.test(trainset.build_testset())
print('biased accuracy on A,', end='    ')
accuracy.rmse(predictions)

#compute the unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings) # testset i now the set B
predictions = algo.test(testset)
print('unbiased accuracy on B,', end='  ')
accuracy.rmse(predictions)

biased accuracy on A,RMSE: 0.8420
unbiased accuracy on B,  RMSE: 0.9475


0.9474531487114454

In [12]:
#untracked